## Really long sentence

In [46]:
import tensorflow as tf
import numpy as np

tf.reset_default_graph()

엄청 긴 문장을 학습시켜보자

"if you want to build a ship, don't drum up people together to "

"collect wood and don't assign them tasks and work, but rather "

"teach them to long for the endless immensity of the sea.")

In [47]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

앞에서 했던것 처럼 문자를 자동적으로 데이터로 쓸 수 있게 만들자

In [48]:
char_set = list(set(sentence)) #사용된 문자만 추출 
print(char_set)
char_dic ={w: i for i, w in enumerate(char_set)}
print(char_dic)

['k', 'l', 'h', ',', 'n', 'y', 'r', 'm', '.', 'i', ' ', 'p', 't', 's', 'c', 'g', 'e', 'u', 'b', 'a', 'f', "'", 'w', 'o', 'd']
{'k': 0, 'l': 1, 'h': 2, ',': 3, 'n': 4, 'y': 5, 'r': 6, 'm': 7, '.': 8, 'i': 9, ' ': 10, 'p': 11, 't': 12, 's': 13, 'c': 14, 'g': 15, 'e': 16, 'u': 17, 'b': 18, 'a': 19, 'f': 20, "'": 21, 'w': 22, 'o': 23, 'd': 24}


그러면 이제 rnn모델을 위한 변수를 정할 수 있다.

한번에 학습시키기에는 너무 길어서 10 문자열마다 짤라서 학습시키자

In [49]:
data_dim = len(char_set)
rnn_hidden_size = len(char_set)
num_classes = len(char_set)
print( len(char_set))

sequence_length = 10  # Any arbitrary number
learning_rate = 0.1

25


이제 데이터를 만들자

10글자 마다 짜르므로 10글자 짜르고 한칸 옆으로 옮겨서 다시 10글자 짜른것을 반복한것이 데이터가 될 것이다.

In [50]:
print('전체 글자수:',len(sentence))

print('나누기로 한 글자수',sequence_length)

print("따라서 만들어야할 데이터의 수:",len(sentence)-sequence_length)

전체 글자수: 180
나누기로 한 글자수 10
따라서 만들어야할 데이터의 수: 170


따라서 10칸이 마지막에 도달했을때는 전체 글자수 180에서 10칸을 뺄때까지 반복한다.

즉 170번 반복한다.

이것을 for문을 사용에 데이터에 자동으로 들어가게끔 설정해놓자

In [51]:
dataX=[]
dataY=[]
for i in range(0,len(sentence)-sequence_length):
    x_str = sentence[i:i + sequence_length] #1부터 11, 2부터 12, 3부터 13....
    y_str = sentence[i + 1: i + sequence_length + 1] #입력의 다음 글자이므로 입력값 보다 1씩 크다
    print(i, x_str, '->',y_str)
    
    x=[char_dic[c] for c in x_str]
    y=[char_dic[c] for c in y_str]
    
    dataX.append(x)
    dataY.append(y)
    
batch_size = len(dataX)
print(batch_size)

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

이전까지는 한 단어 또는 한문장을 학습시켜서 batch size가 1이였지만, 이제는 x data가 여러개라 x data수만큼 batch size를 지정해줘야한다.

이제 rnn cell의 모델을 세우자

문장을 학습할려면 cell이 한층으로는 부족함 우리는 2층으로 쌓자

In [52]:
X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])

X_one_hot = tf.one_hot(X, num_classes)

cell = tf.contrib.rnn.BasicLSTMCell(num_units=rnn_hidden_size, state_is_tuple=True)
layer=2 # cell 층의 수
cell = tf.contrib.rnn.MultiRNNCell([cell]*layer,state_is_tuple= True)

initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell,X_one_hot,initial_state=initial_state, dtype=tf.float32)

## Softmax (FC) in Deep CNN

rnn에서 나온 값을 그대로 쓰지말고 softmax 같은 것을 붙여주자!


In [53]:
X_for_softmax=tf.reshape(outputs,[-1,hidden_size]) # 히든사이즈에 맞게 쌓고 나머진 알아서

softmax_w =tf.get_variable("softmax_w",[hidden_size,num_classes]) #(입력사이즈, 아웃 사이즈)
softmax_b =tf.get_variable("softmax_b",[num_classes]) #(아웃 사이즈)

outputs=tf.matmul(X_for_softmax,softmax_w)+softmax_b 

outputs = tf.reshape(outputs,[batch_size,sequence_length,num_classes]) #펼쳐줌, RNN의 아웃풋과 같은 모양임



얼마나 잘 맞는지 알려줄 loss함수를 세우자

In [54]:
weight= tf.ones([batch_size,sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs,targets=Y,weights=weight)
loss = tf.reduce_mean(sequence_loss)
train=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

학습 가즈ㅏㅏㅏ

In [59]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(500):
    _,l,results = sess.run([train, loss,outputs],feed_dict={X:dataX, Y:dataY})
    
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1)
        print(i,j,''.join([char_set[t] for t in index]),l)

0 0 uuuuuuuuu' 3.28215
0 1 uuuuuuuu'u 3.28215
0 2 uuu''''''' 3.28215
0 3 uuu'u''uuu 3.28215
0 4 uu'uu'uuuu 3.28215
0 5 uuuu'uuuuu 3.28215
0 6 uuuuuuuuuu 3.28215
0 7 uuuuuuuuuu 3.28215
0 8 uuuuuuuuuu 3.28215
0 9 uuuuuuuuuu 3.28215
0 10 uuuuuuuuuu 3.28215
0 11 uuuuuuuuuu 3.28215
0 12 uuuuuuuuuu 3.28215
0 13 uuuuuuuuu' 3.28215
0 14 uuuuuuuu'' 3.28215
0 15 uuuuuuuu'' 3.28215
0 16 uuuu'''''' 3.28215
0 17 uuuu'''''' 3.28215
0 18 uu'''''''' 3.28215
0 19 uuu''''''' 3.28215
0 20 uu'''''''' 3.28215
0 21 u''''''''' 3.28215
0 22 uu'''''''' 3.28215
0 23 uuu''''''' 3.28215
0 24 uuu''''''' 3.28215
0 25 uuu''''''u 3.28215
0 26 uu'''''uuu 3.28215
0 27 ''''''''uu 3.28215
0 28 uuuu'uuuuu 3.28215
0 29 uuuuuuuuuu 3.28215
0 30 uuuuuuuuuu 3.28215
0 31 uuuuuuuuuu 3.28215
0 32 'uuuuuuu'' 3.28215
0 33 uuuuuuuuuu 3.28215
0 34 uuuuu''''' 3.28215
0 35 uuuuu''''u 3.28215
0 36 uuuu''''uu 3.28215
0 37 uu'''''uuu 3.28215
0 38 uu''''uuuu 3.28215
0 39 uu'''uuuuu 3.28215
0 40 uuuuuuuuuu 3.28215
0 41 uuuuuuuuuu 3.28215
0 

2 78            3.05413
2 79            3.05413
2 80            3.05413
2 81            3.05413
2 82            3.05413
2 83            3.05413
2 84            3.05413
2 85            3.05413
2 86            3.05413
2 87            3.05413
2 88            3.05413
2 89            3.05413
2 90            3.05413
2 91            3.05413
2 92            3.05413
2 93            3.05413
2 94            3.05413
2 95            3.05413
2 96            3.05413
2 97            3.05413
2 98            3.05413
2 99            3.05413
2 100            3.05413
2 101            3.05413
2 102            3.05413
2 103            3.05413
2 104            3.05413
2 105            3.05413
2 106            3.05413
2 107            3.05413
2 108            3.05413
2 109            3.05413
2 110            3.05413
2 111            3.05413
2 112            3.05413
2 113            3.05413
2 114            3.05413
2 115            3.05413
2 116            3.05413
2 117            3.05413
2 118            3.054

6 75            2.83126
6 76 o          2.83126
6 77            2.83126
6 78 o          2.83126
6 79            2.83126
6 80 o          2.83126
6 81 o          2.83126
6 82 o          2.83126
6 83 oo         2.83126
6 84 o          2.83126
6 85            2.83126
6 86            2.83126
6 87            2.83126
6 88            2.83126
6 89            2.83126
6 90 o          2.83126
6 91 oo         2.83126
6 92 o          2.83126
6 93            2.83126
6 94            2.83126
6 95            2.83126
6 96 oo         2.83126
6 97 o          2.83126
6 98            2.83126
6 99            2.83126
6 100            2.83126
6 101            2.83126
6 102 o          2.83126
6 103            2.83126
6 104 o          2.83126
6 105            2.83126
6 106 o          2.83126
6 107 o          2.83126
6 108 o          2.83126
6 109            2.83126
6 110            2.83126
6 111 o          2.83126
6 112 o          2.83126
6 113 o          2.83126
6 114            2.83126
6 115 oo         2.83126


9 168            2.72735
9 169            2.72735
10 0            2.68181
10 1            2.68181
10 2            2.68181
10 3            2.68181
10 4            2.68181
10 5            2.68181
10 6            2.68181
10 7            2.68181
10 8            2.68181
10 9            2.68181
10 10   o        2.68181
10 11            2.68181
10 12            2.68181
10 13            2.68181
10 14            2.68181
10 15            2.68181
10 16            2.68181
10 17            2.68181
10 18            2.68181
10 19            2.68181
10 20          , 2.68181
10 21         ,, 2.68181
10 22        ,,, 2.68181
10 23        ,   2.68181
10 24            2.68181
10 25            2.68181
10 26            2.68181
10 27            2.68181
10 28            2.68181
10 29            2.68181
10 30            2.68181
10 31            2.68181
10 32            2.68181
10 33            2.68181
10 34            2.68181
10 35            2.68181
10 36            2.68181
10 37            2.68181
10 38     

13 114            2.49998
13 115  o       e 2.49998
13 116         ee 2.49998
13 117        oe  2.49998
13 118       oo   2.49998
13 119      oe    2.49998
13 120     oo     2.49998
13 121    oo    e 2.49998
13 122   oo    ee 2.49998
13 123  oo    ee  2.49998
13 124       ee   2.49998
13 125      ee    2.49998
13 126     oe   e 2.49998
13 127    oe   eo 2.49998
13 128   oe    oo 2.49998
13 129  oe   eooe 2.49998
13 130       ooee 2.49998
13 131      ooee  2.49998
13 132     ooee   2.49998
13 133    ooee    2.49998
13 134  ooooo     2.49998
13 135  ooe       2.49998
13 136  oo        2.49998
13 137   o        2.49998
13 138          e 2.49998
13 139          e 2.49998
13 140         e  2.49998
13 141       ee   2.49998
13 142   o  ee    2.49998
13 143     oe     2.49998
13 144    oe      2.49998
13 145   oee      2.49998
13 146  oe        2.49998
13 147          s 2.49998
13 148         s  2.49998
13 149        s   2.49998
13 150       s    2.49998
13 151            2.49998
13 152      

...

482 121 m toach th 0.228826
482 122  toach the 0.228826
482 123 toach them 0.228826
482 124  ach them  0.228826
482 125 mch them t 0.228826
482 126 nh them to 0.228826
482 127 t them to  0.228826
482 128 ethem to l 0.228826
482 129 toem to lo 0.228826
482 130  em to lon 0.228826
482 131 em to long 0.228826
482 132 m to long  0.228826
482 133  ta long f 0.228826
482 134 to long fo 0.228826
482 135   long for 0.228826
482 136 nlong for  0.228826
482 137 tong for t 0.228826
482 138 eng for th 0.228826
482 139 n' for the 0.228826
482 140 d for the  0.228826
482 141  for the e 0.228826
482 142 tor the en 0.228826
482 143  r the end 0.228826
482 144 nkthe endl 0.228826
482 145  toe endle 0.228826
482 146 toemendles 0.228826
482 147  emendless 0.228826
482 148 emendless  0.228826
482 149 mendless i 0.228826
482 150 tndless im 0.228826
482 151 msless imm 0.228826
482 152 d ess imme 0.228826
482 153  ess immen 0.228826
482 154 e s immens 0.228826
482 155 ms immensi 0.228826
482 156 iiimmensit 0

486 57  to collec 0.228861
486 58 to lollect 0.228861
486 59   lollect  0.228861
486 60 nlollect w 0.228861
486 61 tollect wo 0.228861
486 62 tllect woo 0.228861
486 63 nlect wood 0.228861
486 64 eect wood  0.228861
486 65 ect wood a 0.228861
486 66 mt wood an 0.228861
486 67 t wood and 0.228861
486 68  aood and  0.228861
486 69 tord and d 0.228861
486 70 ord and do 0.228861
486 71 nd and don 0.228861
486 72 n and don' 0.228861
486 73  and don't 0.228861
486 74 tnd won't  0.228861
486 75 nd won't a 0.228861
486 76 d won't as 0.228861
486 77  aon't ass 0.228861
486 78 ton't assi 0.228861
486 79  n't assig 0.228861
486 80 n't assign 0.228861
486 81 dt assign  0.228861
486 82 t assign t 0.228861
486 83  assign th 0.228861
486 84 tnsign the 0.228861
486 85 nsign them 0.228861
486 86 iign them  0.228861
486 87 itn them t 0.228861
486 88 mn them ta 0.228861
486 89   them tas 0.228861
486 90 dthem task 0.228861
486 91 toem tasks 0.228861
486 92  em tasks  0.228861
486 93 em tasks a 0.228861
4

490 15 uild a shi 0.2292
490 16 pld a ship 0.2292
490 17 md a ship, 0.2292
490 18 e a ship,  0.2292
490 19  anship, d 0.2292
490 20 tnship, do 0.2292
490 21 nship, don 0.2292
490 22 thip, don' 0.2292
490 23 iip, don't 0.2292
490 24 ep, don't  0.2292
490 25 m, don't d 0.2292
490 26 e don't dr 0.2292
490 27  don't dru 0.2292
490 28 ton't arum 0.2292
490 29  n't arum  0.2292
490 30 n't arum u 0.2292
490 31 dt arum up 0.2292
490 32 t arum up  0.2292
490 33  arum up p 0.2292
490 34 toum up pe 0.2292
490 35  um up peo 0.2292
490 36  m up peop 0.2292
490 37 p up peopl 0.2292
490 38  tp people 0.2292
490 39 tp people  0.2292
490 40 p people t 0.2292
490 41 epeople to 0.2292
490 42 teople tog 0.2292
490 43 eople toge 0.2292
490 44 nple toget 0.2292
490 45 nle togeth 0.2292
490 46 ee togethe 0.2292
490 47 e together 0.2292
490 48 neogether  0.2292
490 49 to ether t 0.2292
490 50   ether to 0.2292
490 51 nether to  0.2292
490 52 nther to c 0.2292
490 53 nher to co 0.2292
490 54  em to col 0.2292


493 136  bong for  0.230132
493 137 tong for t 0.230132
493 138 eng for th 0.230132
493 139  ' for the 0.230132
493 140 d for the  0.230132
493 141  for the e 0.230132
493 142 tor the en 0.230132
493 143 or the end 0.230132
493 144   the endl 0.230132
493 145  the endle 0.230132
493 146 toemendles 0.230132
493 147 herendless 0.230132
493 148 erendless  0.230132
493 149 rtndless i 0.230132
493 150 tndless im 0.230132
493 151 rdless imm 0.230132
493 152 d ess imme 0.230132
493 153  ess immen 0.230132
493 154 ess immens 0.230132
493 155 rs immensi 0.230132
493 156   immensit 0.230132
493 157  immensity 0.230132
493 158 tmmensity  0.230132
493 159 gmensity o 0.230132
493 160  ensity of 0.230132
493 161  nsity of  0.230132
493 162 rdity of t 0.230132
493 163 dity of th 0.230132
493 164  gy of the 0.230132
493 165 gy of the  0.230132
493 166 h of the s 0.230132
493 167 oof the se 0.230132
493 168 tf the sea 0.230132
493 169   the sea. 0.230132
494 0 f you want 0.229944
494 1  you want  0.229

497 75 nd won't a 0.228956
497 76 d won't as 0.228956
497 77  aon't ass 0.228956
497 78 ton't assi 0.228956
497 79  n't assig 0.228956
497 80 n't assign 0.228956
497 81 dt assign  0.228956
497 82 t assign t 0.228956
497 83  assign th 0.228956
497 84 tnsign the 0.228956
497 85 nsign them 0.228956
497 86 iign them  0.228956
497 87 itn them t 0.228956
497 88 mn them ta 0.228956
497 89   them tas 0.228956
497 90 dthem task 0.228956
497 91 toem tasks 0.228956
497 92  em tasks  0.228956
497 93 em tasks a 0.228956
497 94 m tasks an 0.228956
497 95  tasks and 0.228956
497 96 tosks and  0.228956
497 97  sks and w 0.228956
497 98 nss and wo 0.228956
497 99 is and wor 0.228956
497 100 s and work 0.228956
497 101 iind work, 0.228956
497 102 tnd work,  0.228956
497 103 nd work, b 0.228956
497 104 d work, bu 0.228956
497 105  aork, but 0.228956
497 106 tork, but  0.228956
497 107 ork, but r 0.228956
497 108 n , but ra 0.228956
497 109  , but rat 0.228956
497 110 s but rath 0.228956
497 111  dut rath

학습이 끝났다면 결과를 출력해보자

In [61]:
results = sess.run(outputs, feed_dict={X: dataX})
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]],end='')

g you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.

## 참고 사이트 
1. https://github.com/hunkim/DeepLearningZeroToAll/blob/master/lab-12-4-rnn_long_char.py
2. https://www.youtube.com/watch?v=2R6nfCNNz1U&list=PLlMkM4tgfjnLSOjrEJN31gZATbcj_MpUm&index=44
3. https://www.youtube.com/watch?v=vwjt1ZE5-K4&list=PLlMkM4tgfjnLSOjrEJN31gZATbcj_MpUm&index=45